<a href="https://colab.research.google.com/github/felixsimard/comp551-p3/blob/main/Hassan_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pickle
import torch
import math
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset, Dataset
from google.colab import drive
from sklearn import preprocessing
from PIL import Image
from typing import List
from datetime import datetime

In [2]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


## Utility Functions

* Displays an image given a bit-based input

In [23]:
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(img)
    # plt.imshow(np.transpose(npimg, (1, 2, 0))) 
    plt.show()

* Class to create datasets with correct dimensionality properties, retreived from StackOverflow

In [24]:
# Reference: https://stackoverflow.com/questions/44429199/how-to-load-a-list-of-numpy-arrays-to-pytorch-dataset-loader
class MyDataset(Dataset):
    def __init__(self, data, targets=None, transform=None, transform_target=None):
        self.data = torch.from_numpy(data).float()
        self.targets = torch.from_numpy(targets).float() if targets is not None else None
        self.transform = transform
        self.transform_target = transform_target
        
    def __getitem__(self, index):
        x = self.data[index]
        y = np.zeros(36, dtype=float)# self.targets[index]

        if self.targets is not None:
            y = self.targets[index]
        else:
            None
        if self.transform:
            x = self.transform(x)
        if self.transform_target:
            y = self.transform_target(y)
        
        return x, y
    
    
    def __len__(self):
        return len(self.data)

* Export a CSV file in the parent directory given a pandas dataframe

In [25]:
def makeMyCSV(dataf: pd.DataFrame)->None:
  filename = 'kaggle_g19_{}.csv'.format(datetime.now())
  dataf.to_csv(filename, sep=',', float_format='{:36}', index=False)

In [26]:
# Felix's load data fn
# Function to return pickle loaded file in an ndarray
def load_data(filename, data_path='/content/drive/MyDrive/data/'):
    drive.mount("/content/drive")
    loaded_pkl = None
    try:
        pkl_buffered = open(data_path+''+filename,'rb')
        loaded_pkl = pickle.load(pkl_buffered)
    except Exception as e:
        print("Error loading data: {}".format(e))
    return loaded_pkl

In [27]:
def get_label_value(labels):
  """
  This function will return a string representing the label of a picture given
  the array label as input:
  Ex ouput: '1a', '4z' ...
  """
  label_temp = labels.tolist()
  label_temp = [int(x) for x in label_temp]
  number = label_temp[:10].index(1)
  letter = alpha_dict[label_temp[10:].index(1)]

  return str(number) + str(letter)

In [28]:
def transform_output(scores):
    """
    Input a Tensor and output will be another Tensor with same dimension but with all elements 0 except two.
    Those 2 elements will have value of 1 and will correspond to the models prediction about which letter and number
    is in the image.
    :param scores:
    :return:
    """
    return_array = []
    score_list = scores.tolist()

    for score in score_list:
        numbers = score[:10]
        letters = score[10:]
        test = lambda x, max_value : 1 if x >= max_value else 0

        new_numbers = [test(x, max(numbers)) for x in numbers]
        new_letters = [test(x, max(letters)) for x in letters]

        return_array.append(new_numbers + new_letters)

    return return_array

In [29]:
def convert_outputs_to_preds(outputs):
    preds = np.empty(shape=(len(outputs), 36))
    for i, output in enumerate(outputs):
        pred = np.zeros(36)
        digit_index = np.argmax(output[:11])
        letter_index = np.argmax(output[11:]) + 11
        pred[digit_index], pred[letter_index] = 1, 1
        preds[i] = pred
    return preds

In [30]:
def correct_digit(pred, label):
    return np.array_equal(pred[:11],label[:11])

def correct_letter(pred, label):
    return np.array_equal(pred[11:],label[11:])

def get_accuracy(results):
    return round(sum(results) / len(results), 2)

## Pickle Data to Numpy NDArray

In [31]:
# loading all data
train_features = load_data("images_l.pkl")[:, None]
train_labels = load_data("labels_l.pkl")
test = load_data("images_test.pkl")[:, None]
train_unlabelled = load_data("images_ul.pkl")[:, None]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
print(train_features.shape, train_features[:1])
print(train_labels.shape, train_labels[:1])

(30000, 1, 56, 56) [[[[  0.   0.   0. ... 175.   0.   0.]
   [  0.   0.   0. ...   0.   0.   0.]
   [  0.   0.   0. ...   0. 175.   0.]
   ...
   [  0.   0.   0. ...   0.   0.   0.]
   [  0.   0.   0. ...   0.   0.   0.]
   [  0.   0.   0. ...   0.   0.   0.]]]]
(30000, 36) [[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


- `train_features` has 30,000 samples of 56x56 images
- `train_labels` labels of the 56x56 images, a 36-bit binary vector
- The code block below verifies the image data are all in numpy n-dimensional arrays, `np.ndarray`

In [33]:
for data in [train_features, train_labels, train_unlabelled, test]:  
  print(type(data) is np.ndarray)

True
True
True
True


## Hyperparameters

In [22]:
epochs = 4
batch = 8
lr = 0.002             # learning rate
channels = 1           # Image input channel
beta = 0.6             # SGD momentum coefficient
train_test_split = 0.3 # 70% training data, 30% validation data
flatten_dim = 56 * 56  # image dim = 56 x 56 px

## Training & Validation Split

In [34]:
# Data transformation parameters
mean = (0.5,)
std = (0.5,)
Transform = transforms.Compose([transforms.Normalize(mean=mean, std=std)])

In [35]:
split_index = math.floor(len(train_labels)*train_test_split)

full_train_l = train_features
val_l = train_features[:split_index]
train_l = train_features[split_index:]

full_train_labels_l = train_labels
val_labels_l = train_labels[:split_index]
train_labels_l = train_labels[split_index:]

print(full_train_l.shape, full_train_labels_l.shape)
print(train_l.shape, train_labels_l.shape)
print(val_l.shape, val_labels_l.shape)

(30000, 1, 56, 56) (30000, 36)
(21000, 1, 56, 56) (21000, 36)
(9000, 1, 56, 56) (9000, 36)


## Tensor DataLoader & Feature Labels

In [36]:
# DataLoaders
all_training = DataLoader(MyDataset(full_train_l, full_train_labels_l, 
                                                   transform=Transform), shuffle=True, batch_size=batch)
training = DataLoader(MyDataset(train_l, train_labels_l, 
                                                   transform=Transform), shuffle=True, batch_size=batch)
validation = DataLoader(MyDataset(val_l, val_labels_l, 
                                                   transform=Transform), shuffle=True)

# Test set for Kaggle
test_labels_ul = np.zeros(len(test))
testing = DataLoader(MyDataset(test, test_labels_ul, 
                                                    transform=Transform), batch_size=batch, shuffle=False)

print(len(all_training)*batch,len(training)*batch, len(testing)*batch)

30000 21000 15000


- The classification task calls for classifying an image that contains:
1. Characters `A-Z` OR `a-z`
2. Numbers `0-9`
- Each image will include any combination of 1 lower OR uppercase character and one number
- Therefore, the labels will have to include every combination of these characters and numbers:
1. 260 different classes: `0-9` AND `A-Z`
2. 260 different classes: `0-9` AND `a-z`
- A total of 520 `labels`

## Conv. NN Class (Implementation of VGG11 Deep CNN)

In [37]:
class CNN(nn.Module):

  # Constructor
  def __init__(self, in_channels=1, num_classes=36):
    super(CNN, self).__init__()         # Access methods in parent class
    self.in_channels = in_channels
    self.num_classes = num_classes
    # convolutional layers 
    self.conv_layers = nn.Sequential(
      nn.Conv2d(self.in_channels, 64, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2),
      nn.Conv2d(64, 128, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2),
      nn.Conv2d(128, 256, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.Conv2d(256, 256, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2),
      nn.Conv2d(256, 512, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.Conv2d(512, 512, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2),
      nn.Conv2d(512, 512, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.Conv2d(512, 512, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2)
      )
        # fully connected linear layers
    self.linear_layers = nn.Sequential(
      nn.Linear(in_features=512, out_features=4096),
      nn.ReLU(),
      nn.Dropout2d(0.5),
      nn.Linear(in_features=4096, out_features=4096),
      nn.ReLU(),
      nn.Dropout2d(0.5),
      nn.Linear(in_features=4096, out_features=self.num_classes)
      )
    
  def forward(self, x):
      x = self.conv_layers(x)
      # flatten to prepare for the fully connected layers
      x = x.view(x.size(0),-1)
      x = self.linear_layers(x)
      return x
  

# Model Training with CUDA



In [38]:
model = CNN().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=beta)
steps = len(training)

In [39]:
for epoch in range(epochs):
  running_loss = 0.0
  model.train()
  for i, (inputs, targets) in enumerate(training):
    inputs, targets = inputs.to(device), targets.to(device)
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs,targets)

    loss.backward()
    optimizer.step()
    running_loss += loss.item()
  
    if i % 100 == 99:    # print every 100 mini-batches
      print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
      running_loss = 0.0

[1,   100] loss: 0.448
[1,   200] loss: 0.230
[1,   300] loss: 0.223
[1,   400] loss: 0.221
[1,   500] loss: 0.219
[1,   600] loss: 0.220
[1,   700] loss: 0.217
[1,   800] loss: 0.218
[1,   900] loss: 0.218
[1,  1000] loss: 0.217
[1,  1100] loss: 0.215
[1,  1200] loss: 0.217
[1,  1300] loss: 0.215
[1,  1400] loss: 0.215
[1,  1500] loss: 0.215
[1,  1600] loss: 0.215
[1,  1700] loss: 0.214
[1,  1800] loss: 0.214
[1,  1900] loss: 0.213
[1,  2000] loss: 0.213
[1,  2100] loss: 0.215
[1,  2200] loss: 0.213
[1,  2300] loss: 0.214
[1,  2400] loss: 0.213
[1,  2500] loss: 0.213
[1,  2600] loss: 0.213
[2,   100] loss: 0.213
[2,   200] loss: 0.212
[2,   300] loss: 0.213
[2,   400] loss: 0.213
[2,   500] loss: 0.213
[2,   600] loss: 0.212
[2,   700] loss: 0.213
[2,   800] loss: 0.213
[2,   900] loss: 0.213
[2,  1000] loss: 0.213
[2,  1100] loss: 0.213
[2,  1200] loss: 0.212
[2,  1300] loss: 0.212
[2,  1400] loss: 0.212
[2,  1500] loss: 0.212
[2,  1600] loss: 0.212
[2,  1700] loss: 0.212
[2,  1800] 

In [41]:
# Update accuracy metric on validation set
validation_acc = []
model.eval()
digit_results = []
letter_results = []
for i, data in enumerate(validation):
    val_inputs, val_labels = data
    val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)
    outputs = model(val_inputs)
    val_labels = val_labels.cpu().detach().numpy()
    preds = convert_outputs_to_preds(outputs.cpu().detach().numpy())
    digit_results.append(correct_digit(preds[0], val_labels[0]))
    letter_results.append(correct_letter(preds[0], val_labels[0]))
digit_accuracy = get_accuracy(digit_results)
letter_accuracy = get_accuracy(letter_results)
total_accuracy = get_accuracy(digit_results and letter_results)
validation_acc.append((digit_accuracy, letter_accuracy, total_accuracy))

# Check validation accuracy
for i, val_accuracy in enumerate(validation_acc):
  print('Epoch = {}, Total Acc = {}, Digit Acc = {}, Letter Acc = {}'.format(i+1, val_accuracy[2], val_accuracy[0], val_accuracy[1]))

Epoch = 1, Total Acc = 0.04, Digit Acc = 0.1, Letter Acc = 0.04


In [ ]:
df = pd.DataFrame(columns=['# Id', 'Category'])
#device = torch.device('cpu')
with torch.no_grad():
    i = 0
    for data in test_ul_dataloader:
        inputs = inputs.to(device) 
        targets = targets.to(device)
        outputs = model(inputs)
        predictions = transform_output(outputs)
        for pred in predictions:
            label = ''.join(str(x) for x in pred)
            df.loc[i] = [i, label]
            i += 1

df = df.iloc[:15001]

df

In [ ]:
makeMyCSV(df)